right now, this compares 9 channel CNN vs. GLMs, in terms of raw performance (cc^2 or cc), on ALL neurons (so no subset stuff yet)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from collections import OrderedDict
from itertools import product
import h5py
import numpy as np
import pandas as pd

In [3]:
from tang_jcompneuro.model_fitting_postprocess import load_model_performance

In [4]:
def check_all():
    subsets_to_check = ('all', 'OT')
    datasets_to_check = ('MkA_Shape', 'MkE2_Shape')
    
    models_to_examine = [
        ('cnn', 'b.1'),
        ('cnn', 'b.2'),
        ('cnn', 'b.3'),
        ('cnn', 'b.6'),
        ('cnn', 'b.9'),
        ('glm', 'linear_poisson'),
        ('glm', 'fpower_poisson'),
        ('glm', 'gqm.2_poisson'),
        ('glm', 'gqm.4_poisson'),
        ('glm', 'gqm.8_poisson'),
        ('glm', 'linear_gaussian'),
        ('glm', 'fpower_gaussian'),
        ('glm', 'gqm.2_gaussian'),
        ('glm', 'gqm.4_gaussian'),
        ('glm', 'gqm.8_gaussian'),
    ] # TODO should make this a ordered dict.
      # so that it will be easier to collect best val model for each model class.
    
    score_all = []
    for dataset, subset in product(datasets_to_check, subsets_to_check):
        
        print(dataset, subset)
        for (model_type, model_subtype) in models_to_examine: 
            for percent in (25, 50, 75, 100):
                score_new_cc = np.asarray(
                    [load_model_performance(dataset, subset, percent, s, model_type, model_subtype)['corr'] for s in range(2)]
                ).mean(axis=0)

                score_new_cc2 = np.asarray(
                    [load_model_performance(dataset, subset, percent, s, model_type, model_subtype)['corr']**2 for s in range(2)]
                ).mean(axis=0)

                score_all.append({
                    'dataset': dataset,
                    'subset': subset,
                    'model': model_type + '_' + model_subtype,
                    'train_set_percentage': percent,
                    'cc': score_new_cc.mean(),
                    'cc2': score_new_cc2.mean(),
                    # later on, I can add neuron subset, etc.
                })
            
    score_all = pd.DataFrame(score_all,
                             columns = ['dataset', 'subset', 'model',
                                        'train_set_percentage', 'cc', 'cc2']).set_index(['dataset', 'subset', 'model', 'train_set_percentage'], verify_integrity=True)
    return score_all

In [5]:
df_all = check_all()

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [6]:
df_all.columns.name = 'metric'

In [7]:
# this is the way to change a multiindex's names.
# df_all.index.rename(['haha','haha2','haha3'], inplace=True)

In [8]:
df_all.unstack('subset').unstack('train_set_percentage')

metric                                cc                                \
subset                                OT                                 
train_set_percentage                 25        50        75        100   
dataset    model                                                         
MkA_Shape  cnn_b.1              0.203596  0.255400  0.284531  0.300833   
           cnn_b.2              0.222059  0.278523  0.303653  0.319774   
           cnn_b.3              0.228400  0.282121  0.311115  0.326022   
           cnn_b.6              0.236278  0.290902  0.318078  0.333677   
           cnn_b.9              0.240418  0.295547  0.321147  0.337550   
           glm_fpower_gaussian  0.196035  0.233797  0.249867  0.260039   
           glm_fpower_poisson   0.197372  0.233346  0.251369  0.261663   
           glm_gqm.2_gaussian   0.152520  0.191270  0.214334  0.221168   
           glm_gqm.2_poisson    0.156291  0.198711  0.223213  0.231374   
           glm_gqm.4_gaussian   0.153093  0.200406  0.224381  0.235860   
           glm_gqm.4_poisson    0.157559  0.207038  0.232276  0.244452   
           glm_gqm.8_gaussian   0.144163  0.192944  0.217162  0.234528   
           glm_gqm.8_poisson    0.144598  0.195143  0.221188  0.237862   
           glm_linear_gaussian  0.147168  0.171281  0.184606  0.193430   
           glm_linear_poisson   0.153542  0.175858  0.192788  0.201861   
MkE2_Shape cnn_b.1              0.236749  0.309746  0.344958  0.370987   
           cnn_b.2              0.266365  0.343424  0.375688  0.401992   
           cnn_b.3              0.279707  0.349981  0.384757  0.407829   
           cnn_b.6              0.294293  0.365443  0.397756  0.424149   
           cnn_b.9              0.298323  0.366810  0.398881  0.424945   
           glm_fpower_gaussian  0.224667  0.268426  0.289003  0.312055   
           glm_fpower_poisson   0.223642  0.267847  0.288741  0.309706   
           glm_gqm.2_gaussian   0.180987  0.246647  0.275221  0.295394   
           glm_gqm.2_poisson    0.178064  0.257111  0.286107  0.307597   
           glm_gqm.4_gaussian   0.179890  0.249780  0.283408  0.302531   
           glm_gqm.4_poisson    0.177469  0.252427  0.289912  0.309054   
           glm_gqm.8_gaussian   0.162252  0.237284  0.270367  0.292490   
           glm_gqm.8_poisson    0.156002  0.234865  0.271772  0.295629   
           glm_linear_gaussian  0.184443  0.226468  0.244978  0.248554   
           glm_linear_poisson   0.189885  0.238631  0.259594  0.262213   

metric                                                                  \
subset                               all                                 
train_set_percentage                 25        50        75        100   
dataset    model                                                         
MkA_Shape  cnn_b.1              0.297015  0.333807  0.347550  0.356081   
           cnn_b.2              0.329111  0.371807  0.389945  0.399007   
           cnn_b.3              0.339586  0.384123  0.401758  0.412187   
           cnn_b.6              0.352042  0.396253  0.414610  0.427011   
           cnn_b.9              0.358033  0.400676  0.420362  0.432107   
           glm_fpower_gaussian  0.249654  0.272533  0.281365  0.289380   
           glm_fpower_poisson   0.248455  0.274507  0.283640  0.292536   
           glm_gqm.2_gaussian   0.247768  0.288095  0.307894  0.322815   
           glm_gqm.2_poisson    0.257239  0.301228  0.322415  0.338436   
           glm_gqm.4_gaussian   0.261707  0.303113  0.323319  0.337893   
           glm_gqm.4_poisson    0.271202  0.315092  0.336562  0.352186   
           glm_gqm.8_gaussian   0.257300  0.297860  0.318724  0.333564   
           glm_gqm.8_poisson    0.265291  0.308186  0.330209  0.346025   
           glm_linear_gaussian  0.195722  0.212475  0.220703  0.225797   
           glm_linear_poisson   0.203461  0.221741  0.230492  0.236187   
MkE2_Shape cnn_b.1              0.357094  0.409471  0.436075  0.448963   
   